- 해당 프로그램 코딩은 임형욱이 개발 했으며 사용자 허락 없이 절대 상업용으로 사용해선 안됩니다.
- 또한 이 프로그램으로 특정 회사에 피해를 줄 경우
- 모든 책임은 프로그램 사용자에게 있음을 미리 공지합니다.
- 이 프로그램과 관련한 문의는 hwlim2010@naver.com으로 보내주세요^^

In [61]:
# 경남신문 창원뉴스 기사 크롤링하기
# http://www.knnews.co.kr/

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy 
import pandas as pd  
import xlwt
import random
import os   # 폴더 생성하기 위해 필요함
import requests
import re

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print("경남신문 창원뉴스 기사 크롤링하기")
print("=" *100)
query = input('1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): ')

query_txt = '경남신문' # 파일 저장 이름용 변수

cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 12) # 한 페이지에 10-건의 기사가 보여짐
print("크롤링 할 총 페이지 번호: ",page_cnt)

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 ★★★★★★
f_dir = input("3.파일을 저장할 폴더명만 쓰세요(사용자 에게 맞는 절대경로 입력하세요.):")
if f_dir == '' :
    f_dir="c:\\Users\\"

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'
    
#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/Users/pc/chromedriver/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get('http://www.knnews.co.kr/')
driver.maximize_window() 
time.sleep(2)

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element_by_id("textfield")
driver.find_element_by_id("textfield").click( )
element.send_keys(query)
element.send_keys("\n")
time.sleep(2)

#Step 6.뉴스 더보기 선택하기
driver.find_element_by_link_text('더보기').click()
time.sleep(2)

#Step 7. 각 뉴스기사의 상세 정보를 수집합니다

click_count = 1

# 각 뉴스기사의 URL 주소를 추출하여 목록을 먼저 만든 후 상세 크롤링을 진행합니다.
# URL 수집하여 목록 만들기
url_list=[]
count = 0
#while ( click_count <= page_cnt + 1)  :
for a in range(1,page_cnt+1) :
    
    print("%s 번째 페이지에서 기사의 URL 정보를 추출합니다==============" %a)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    result_1 = soup.find('div','se_cont1')
    result_url_list=result_1.find('ul').find_all('li')
        
    for i in result_url_list:
        url = i.find('a')['href'][5:]
        print(url)
        full_url = 'http://www.knnews.co.kr/'+ url
        url_list.append(full_url)
            
        count += 1

        if count == cnt :
            break  

    print("%s 번째 페이지까지 %s건 기사 수집 완료 ====================" %(a,count))
    print("\n")
    
    a += 1
    try :
        driver.find_element_by_link_text("%s" %a).click()
    except :
        driver.find_element_by_link_text('다음').click()
        
    time.sleep(2)

# Step 8. 각 뉴스 별 상세 내역을 수집합니다
no = 1

title2=[]
date2=[]
content2=[]

for i in range(0,len(url_list)):  
            
    print("\n")
    print("%s 번째 경남신문 창원 뉴스 상세 정보입니다==========================" %no)

    driver.get(url_list[i])

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    content_all = soup.find('div','center')

    # 뉴스제목 추출
    title = content_all.find('div','cont_tit').find('h1','h1_tex').get_text()
    print('1.뉴스제목:' , title)
    title2.append(title)

    # 뉴스작성일
    e_date = content_all.find('div','cont_tit').find('li','loca_left').get_text()
    print('2.뉴스작성일: ',e_date.split(' ')[2])
    date2.append(e_date.split(' ')[2])
    
    # 뉴스내용
    content = str(content_all.find('div','cont_cont').find_all('p'))
    content1 = re.sub('<.+?>', '', content, 0).strip()
    print('3.뉴스내용: ',content1)
    content2.append(content1)
    
    #driver.back()  # 이전 페이지로 돌아가기 ★★★★★
    
    time.sleep(2)
    
    no += 1

    if no > cnt :
        break

# Step 9. 출력 결과를 저장하기
# 출력 결과를 표(데이터 프레임) 형태로 만들기

gnnews = pd.DataFrame()

gnnews['뉴스작성일'] = pd.Series(date2)
gnnews['뉴스제목'] = pd.Series(title2) 
gnnews['뉴스내용'] = pd.Series(content2)
gnnews['뉴스URL'] = pd.Series(url_list)
       
# csv 형태로 저장하기
gnnews.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
gnnews.to_excel(fx_name , index=False)

driver.close( )

경남신문 창원뉴스 기사 크롤링하기
1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): 창원
2.크롤링 할 건수는 몇건입니까?: 100
크롤링 할 총 페이지 번호:  9
3.파일을 저장할 폴더명만 쓰세요(사용자 에게 맞는 절대경로 입력하세요.):
1 번째 페이지에서 기사의 URL 정보를 추출합니다==============
/news/articleView.php?idxno=1358536
/news/articleView.php?idxno=1358510
/news/articleView.php?idxno=1358529
/news/articleView.php?idxno=1358533
/news/articleView.php?idxno=1358528
/news/articleView.php?idxno=1358513
/news/articleView.php?idxno=1358532
/news/articleView.php?idxno=1358488
/news/articleView.php?idxno=1358495
/news/articleView.php?idxno=1358484
/news/articleView.php?idxno=1358493
/news/articleView.php?idxno=1358478
1 번째 페이지까지 12건 기사 수집 완료 ====================


2 번째 페이지에서 기사의 URL 정보를 추출합니다==============
/news/articleView.php?idxno=1358476
/news/articleView.php?idxno=1358475
/news/articleView.php?idxno=1358474
/news/articleView.php?idxno=1358472
/news/articleView.php?idxno=1358471
/news/articleView.php?idxno=1358469
/news/articleView.php?idxno=1358468
/news/articleView.php?idxno=135



6 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: [사구체신염] 소변이 콜라색? 콩팥이 보내는 이상신호
2.뉴스작성일:  2021-08-23
3.뉴스내용:  [콩팥은 우리 몸 옆구리에 쌍으로 위치하는 장기로, 콩 같은 모양에 팥과 색이 비슷해 콩팥이라고 한다. 콩팥과 신장, 두 용어를 함께 사용하는데 콩팥이 순우리말이라 이해가 쉬우며, 신장은 심장과 발음이 비슷해 혼동을 막고자 콩팥이라는 용어 사용을 권하고 있다. 콩팥은 노폐물과 남는 수분을 배출하고 산과 알칼리·전해질의 균형을 조절하며, 뼈 건강을 유지하고 피를 만드는 등 생명의 유지에 필수적인 장기 중 하나다., 평소와 같이 생활하던 중 소변을 볼 때 갑자기 거품이 심하게 발생하거나 색이 콜라 빛으로 변해 병원을 방문하고 자신의 증상에 대해 당황하며 얘기하는 환자들이 더러 있다. 이처럼 소변에 거품이나 피가 섞여 나타나면 콩팥병의 신호일 수 있다는 것은 의료인이 아니더라도 널리 알려진 의학 상식이 됐다. 이러한 증상이 일시적으로 나타난다면 크게 우려할 필요는 없으나, 증상이 반복되거나 악화되는 경우 콩팥병을 의심해 볼 필요가 있다. 이러한 콩팥병 중 사구체신염에 대해 알아보겠다., 콩팥은 앞서 이야기한 것처럼 우리가 살아가는 데 반드시 필요한 장기이다. 콩팥의 구조 중에서, 보통 소변을 만들어 내고 노폐물을 여과하는 핵심적인 필터 역할을 하는 것이 사구체이다. 사구체는 가느다란 모세혈관들이 실타래처럼 뭉친 형태로 양쪽 콩팥에 각각 약 100만개씩 존재하며, 마치 거름망 같은 역할을 하여 체내에 필요한 혈구 및 단백질 배출을 막아주는 것이 사구체의 주된 기능이다., 사구체신염은 다양한 원인에 의해 콩팥에 있는 사구체에 염증이 발생해 그에 따라 증상이 나타나는 질병군을 뜻한다. 사구체신염은 다른 원인 질환이 없이 사구체 자체에 문제가 생긴 일차성 사구체신염과 다른 질환에 의해 이차적으로 발생한 이차성 사구체신염으로 나눌 수 있다. 통상 면역은 세균이나



12 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: 창원 시민, 한 달간 자전거 출퇴근 참여합시다
2.뉴스작성일:  2021-08-22
3.뉴스내용:  [창원시가 전국 15개 도시와 함께 22일부터 한 달간 자전거 출퇴근으로 탄소 줄이기에 나선다., 시는 교통량 감축으로 온실가스를 줄이는 등 기후위기 대응을 위해 22일 ‘에너지의 날’을 시작으로 오는 9월 22일 ‘세계 차 없는 날’까지 32일간 창원시를 포함한 대구, 인천, 수원, 순천, 전주 등 전국 15개 도시가 동시에 참여해 자전거 출퇴근 비디오 판독 요구를 진행한다고 22일 밝혔다., 이번 비디오 판독 요구는 시민 누구나 참여할 수 있고, 스마트폰 앱에서 에코 바이크를 다운받아 자전거를 탈 때 앱을 실행하면 포인트가 적립된다. 우수 참가 도시와 참가자에게 상패가 주어진다., 이와 별도로 시는 실적이 우수한 참가자 130명에게 1만원 상당의 누비전도 지급하고, 전체 참여자를 대상으로 경품 추첨도 진행할 계획이다., 또 시는 기후위기에 대응해 각종 시민참여 운동을 전개하고 있다. 매월 22일 실시하는 △승용차 대신 대중교통 이용 △탄소포인트제 가입 △그린터치 설치 △누비자(자전거) 이용하기 △채식하기 등 5대 기후 행동 실천과 함께 ‘불필요한 이메일 비우기’ 캠페인 등 저탄소 녹색 생활 실천 분위기 확산에도 적극 노력하고 있다., 자세한 내용은 창원시 홈페이지(changwon.go.kr) 또는 창원시지속가능발전협의회 홈페이지(greencw21.or.kr)를 통해 확인할 수 있다., 한편 창원시는 2020년 자전거 출퇴근 챌린지에서 창원, 광주, 대구, 대전, 수원, 전주 등 12개 도시가 경합해 출퇴근 부문에서 우수참가 도시 1위에 선정된 바 있다., 이현근 기자 san@knnews.co.kr]


13 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: 경남 이달 ‘코로나 사망’ 8명째… 월평균



19 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: 창원 학원강사 “격주 진단검사 가혹”
2.뉴스작성일:  2021-08-22
3.뉴스내용:  [창원시가 사회적 거리두기 4단계를 이달 29일까지 연장한 가운데 학원강사를 대상으로 2주마다 유전자증폭(PCR) 선제검사를 받도록 행정명령하면서 일부 학원강사들이 ‘가혹한 조치’라며 불만을 호소하고 있다., 창원시는 지난 13일 코로나19 감염확산 방지를 위해 학원 종사자와 교습소 종사자 등에게 코로나19 진단 검사를 의무적으로 받도록 행정명령 했다. 이를 위반할 경우 200만원 이하의 벌금과 확진자 발생 시 방역 비용 등 구상권을 청구할 수 있다., 학원강사들은 코로나19 확산을 막기 위한 시의 취지는 이해하지만 학원강사들의 입장을 고려하지 않은 처사라며 실효성에 의문을 제기했다., 수학교습소를 운영하는 신은정(42·여·마산회원구 양덕동)씨는 “평일은 수업 때문에 검사를 받을 수 없어 주말에 검사를 받으러 가면 강사들이 몰리는 경우가 많다”며 “강사들은 혹시 모를 위험에 대비해 동선을 최소화하고 있지만, 학생들은 주말에 놀러 가거나 여행을 가기도 해 강사만 검사를 받는 것이 무슨 의미가 있는지 의문스럽다”고 반문했다. 또 다른 학원강사 A씨는 “백신 접종도 완료했는데 또 검사를 받아야 하는 것이 아이러니하다. 강사들만 조심한다고 확진자가 안 나오는 것도 아닌데 검사 대상자를 특정 짓는 것이 맞는지 모르겠다 ”고 말했다., 이에 대해 창원시는 청소년 확진자 증가의 예방조치로 선제검사를 진행하는 것이라는 입장이다., 창원시 관계자는 “현재 학원 종사자 선제검사 관련해서 민원이 많이 들어와 검사 주기를 늘리는 방향 등 개선 가능한 부분을 논의 중에 있다”며 “청소년 확진자 증가와 개학을 앞둔 시점에 맞춰 예방적 조치로 선제검사를 진행하는 것이기에 학원종사자의 협조를 당부 드린다”고 전했다., 박준영 기자 bk6041@knnews.co.kr]


20 번째 경남신문



26 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: 가을장마 첫날, 경남 평균 96㎜ 물폭탄
2.뉴스작성일:  2021-08-21
3.뉴스내용:  [가을장마가 시작된 21일 도내 강풍을 동반한 많은 비로 곳곳에서 피해가 잇따랐다., 경남도에 따르면 이날 양산시 주남동 공장지역에서는 길이 100m, 높이 15m의 옹벽이 붕괴되면서 주변에 주차된 차량 2대와 시설 일부가 흘러내린 토사에 묻혀 복구가 진행중이다., 창원시 의창구 북면의 한 건물의 지하가 폭우로 인해 침수돼 5t가량 소방차를 동원해 배수지원 활동을 벌였다. 또 창원시 의창구 소계지하차도가 침수돼 안전조치와 배수작업을 벌였다. 창원시 대방동에서는 빌딩의 간판이 일부 떨어지면서 추가 파손 위험이 있어 안전하게 조치를 취했다. 고성군에서는 2층에 세워둔 실외기와 합판이 강풍으로 인해 파손돼 소방당국이 출동하기도 했다., 이날 도내 소방당국은 인원 116명과 장비 35개를 투입해 간판, 낙석 등 안전조치 27건과 침수로 인한 급배수지원 1건 등 28건의 소방활동을 벌인 것으로 나타났다., 한편 21일 오후 6시 현재 도내 내려진 기상특보는 울산·남해·거제·통영에 강풍주의보가, 양산·거제·통영·김해에는 호우주의보가 발효 중이다., 21일 오후 4시 기준 강우량은 고성 179.9㎜, 창원 160.4㎜, 남해 148.5㎜ 등으로, 도내 평균적으로는 96.7㎜로 많은 비가 내린 것으로 나타났다., 이민영 기자 mylee77@knnews.co.kr]


27 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: 21일 코로나 확진자 58명 추가… 20대·80대 2명 숨져
2.뉴스작성일:  2021-08-21
3.뉴스내용:  [경남도는 20일 오후 5시 이후 도내 신종 코로나바이러스 감염증(코로나19) 신규확진자 58명이 발생했다고 21일 밝혔다., 지역별로는 창원 22명, 김해 18명, 진주 6명, 창녕 3명,

...



71 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: 창녕·함안, 외국인 노동자 등 집단감염 확산
2.뉴스작성일:  2021-08-18
3.뉴스내용:  [경남지역 코로나19 누적 확진자가 9000명을 넘어섰다. 지난 8일 8000명 돌파 이후 10일 만에 1000명이 늘어난 것이다. 하루 평균 100명대의 신규 확진자가 쏟아지고 있는 가운데 이러한 추세가 이어지면 이달 중 누적 1만명을 돌파할 것으로 예상된다., 경남도는 17일 오후 5시 이후 도내 코로나19 신규 확진자 107명이 발생했다고 18일 밝혔다., 지역별로는 창원 34명, 김해 17명, 창녕 10명, 양산 9명, 함안 8명, 진주 7명, 거제 5명, 통영 4명, 거창 3명, 고성·의령 각 2명, 사천·밀양·남해·산청·함양·합천 각 1명이다., 감염경로별로는 도내 확진자 접촉이 61명으로 가장 많았다. 세부적으로 가족 간 감염이 24명, 직장 내 감염이 25명, 지인 9명, 동선 3명이다. 이 밖에 타지역 확진자 접촉은 8명, 수도권 관련 1명, 창원소재 마트 관련 2명, 해외 입국자 2명이고, 나머지 33명은 감염경로 조사 중이다., 이날 창녕과 함안 지역 공장에서 집단감염이 잇따라 확인됐다. 창녕 소재 공장에서는 지난 13일 근무자 1명이 확진 판정을 받은 후 총 17명의 확진자가 나왔다. 14일 4명, 16일 1명, 17일 1명, 18일 10명이다. 방역당국은 전체 확진자 중 1명을 제외한 16명이 외국인 노동자이며, 이들이 기숙사 생활을 통해 감염된 것으로 파악하고 있다., 함안 소재 공장에서도 지난 16일 근무자 1명이 최초 확진 판정을 받은 후 현재까지 총 8명의 확진자가 발생했다. 도는 해당 공장에 임시 선별진료소를 설치해 170명에 대한 전수검사를 진행 중이다., 방역당국은 또 확진자 2명이 발생한 의령의 한 회사에 대해서도 임시 선별진료소를 설치해 근무자 120명에 대해 검사를 진행 중이다., 남창원농협 마트 관련 확진자는 2



77 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: 18일 경남 107명 추가…창녕·함안서 공장 집단감염 잇따라
2.뉴스작성일:  2021-08-18
3.뉴스내용:  [경남도는 17일 오후 5시 이후 도내 신종 코로나바이러스 감염증(코로나19) 신규확진자 107명이 발생했다고 18일 밝혔다., 지역별로는 창원 34명, 김해 17명, 창녕 10명, 양산 9명, 함안 8명, 진주 7명, 거제 5명, 통영 4명, 고성·의령·거창 각 2명, 사천·밀양·남해·산청·함양·거창·합천 각 1명이다., 감염경로별로는 도내 확진자 접촉이 61명으로 가장 많았다. 세부적으로 가족 간 감염이 24명, 직장 내 감염이 25명, 지인 9명, 동선 3명이다. 이 밖에 타지역 확진자 접촉은 8명, 수도권 관련 1명, 창원소재 마트 관련 2명, 해외 입국자 2명이고, 나머지 33명은 감염경로 조사 중이다., 이날 창녕과 함안 지역 공장에서 집단감염이 잇따라 확인됐다., 창녕 소재 공장에서는 지난 13일 근무자 1명이 확진 판정을 받은 후 현재까지 총 17명의 확진자가 나왔다. 14일 4명, 16일 1명, 17일 1명, 18일 10명이다. 방역당국은 전체 확진자 중 1명을 제외한 16명이 외국인 노동자이며, 이들이 기숙사 생활을 통해 감염된 것으로 파악하고 있다., 함안 소재 공장에서도 지난 16일 근무자 1명이 최초 확진 판정을 받은 후 현재까지 총 8명의 확진자가 발생했다. 도낸 해당 공장에 임시 선별진료소를 설치해 170명에 대한 전수검사를 진행 중이다.,  , 남창원 농협 관련 확진자도 2명이 추가돼 누적 확진자는 총 70명으로 늘었다., 또 의령의 한 회사에서도 직장동료 2명이 양성판정을 받았다. 도는 해당 회사에 임시 선별진료소를 설치해 종사자 120명에 대해 검사를 진행 중이다., 이날 오후 5시 기준 도내 코로나19 누적 확진자는 총 9102명이다. 1255명은 입원 중이고, 7824명은 퇴원했고, 23명은 사망자다.,



87 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: “지방 무시하는 일방적 중개보수 개편 반대”
2.뉴스작성일:  2021-08-17
3.뉴스내용:  [국토교통부가 부동산중개보수(중개 수수료) 인하를 추진 중인 가운데 공인중개사들은 정부의 부동산 정책 실패에 대한 국민들의 불만을 공인중개사들에게 떠넘기고 있는 것이며, 전국 11만 공인중개사들의 생존권을 빼앗고 있다며 반발하고 있다., 국토교통부와 국토연구원이 최종 개편안을 내기 전 국민과 협회·학계·시민 등의 다양한 의견을 수렴한다는 취지로 17일 오후 2시 중개보수 및 중개서비스 개선 방안 온라인 토론회를 예고하면서 한국공인중개사협회는 이날 청와대와 국회, 더불어민주당사 앞, 국토교통부, 전국시도지부에서 전국 동시 시위를 시작했다. 박용현 협회장은 이날부터 국토교통부 앞에서 분쟁 해결 방안을 마련하고 진정성 있는 협의를 진행하라며 무기한 단식투쟁에 돌입했다. 한국공인중개사협회 경상남도지부도 이날 오전 10시부터 창원시 의창구 용호동 더불어민주당 경남도당 앞에서 정부의 중개보수 개편안을 규탄하는 1인 릴레이 시위를 펼쳤다., 
“부동산 정책실패 중개사에 전가가격구간 지역 특수성 반영안돼전반적 보수 인하는 생존권 위협”
, 국토교통부는 부동산 가격 상승에 따라 중개보수가 증가하면서 국민의 부담이 크게 증가하고 있으며 개선요구가 지속적으로 제기되고 있다고 개편의 당위성을 강조하고 있다. 지금까지 논의된 방안 중에는 2~9억 0.4%, 9~12억 0.5%, 12~14억은 0.6% 15억 이상은 0.7% 요율 상한을 적용하는 안이 유력할 것으로 보고 있다. 소비자들에게 유리한 안의 경우 2~12억원에 0.4%, 12억 이상은 0.7%의 요율 상한을 적용하는 방안까지 나왔다., 공인중개사들은 고가 주택에 대한 중개보수 인하는 동의할 수 있지만 문제가 되지 않는 구간을 포함한 전반적 인하는 공인중개사들의 생존권을 위협하는 일이기에 절대 수용할 수 없다고 강조했다



94 번째 경남신문 창원 뉴스 상세 정보입니다==========================
1.뉴스제목: 창원시, ‘69명 집단감염’ 남창원농협에 책임 문다
2.뉴스작성일:  2021-08-17
3.뉴스내용:  [속보= 창원시가 코로나 확진자가 발생했는데도 숨기고 영업해 69명이 집단감염된 남창원농협에 대해 10일간 운영중단과 2250만원의 과태료를 부과하기로 했다. 또 임시선별검사소 추가 설치와 2만여 시민의 진단 검사비, 확진자 치료및 격리 비용 발생 등에 따른 구상금도 청구하기로 했다.(17일 5면), 허성무 창원시장은 17일 남창원농협발 코로나19와 관련한 브리핑에서 이 같이 밝히고 행정·사법적 대응에 착수했다. 허 시장은 “남창원농협 측의 코로나 방역수칙 위반사례와 위반사항에 따른 처분에 대해 면밀히 검토한 결과, 사회적 거리두기가 2단계로 격상된 7월 15일부터 8월 4일까지 사회적 거리두기 2단계 이상에서 금지돼 있는 집객 행사금지 위반 15건을 확인했다”면서 “감염병예방법에 따라 15건에 150만원씩 총 2250만원의 과태료를 부과하고, 운영 중단 10일도 처분할 계획이다”고 밝혔다. 또 “과태료 부과와 운영중단 처분은 사전통지와 의견 청취 등의 절차를 거친 다음 처리되며, 남창원농협의 자진 휴업기간과는 별개로 부과된다”고 덧붙였다., 시는 행정처분과 별도로 사법적 대응으로 ‘구상금 청구’도 강행할 것이라고 밝혔다. 시는 이번 사태와 관련해 구상금 청구 등 행정·사법적 조치를 취하라는 보건복지부 장관의 지시도 있었고, 이에 따라 선별검사소 추가 설치 비용과 코로나19 진단검사비, 확진자 치료 및 격리비용 등 비용 발생건에 대해 ‘구상금 청구의 소’도 준비하고 있다., 시는 구체적인 구상금 규모는 보다 엄격하고 정밀하게 검토할 예정이지만 지난해 광화문 집회에 참석했다가 확진된 두산공작기계내 매점관련자에 대해 3억원의 구상금 청구 민사소송을 한 것보다는 클 것으로 내다봤다., 하지만 시는 이러한 조치에 대해 남창원농협측에서 농협중앙회와 연대해 

<ipython-input-61-a65908bff084>:163: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  gnnews.to_excel(fx_name , index=False)
